In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPool2D, InputLayer, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
def pick_and_show_image(data, labels):
    i = np.random.randint(0, len(data), 1)[0]
    img = np.squeeze(data[i,:,:])
    y = labels[i]

    figsize=(3,3)
    plt.figure(figsize=figsize)
    plt.imshow(img, cmap="gray_r")
    plt.title("Label : {}".format(y))
    plt.axis('off')
    plt.show()
    return img, y

## Read mnist data

In [ ]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.mnist.load_data()

In [ ]:
print("training set shape: {}".format(x_train_full.shape))
print("training set label shape: {}".format(y_train_full.shape))
print("test set shape: {}".format(x_test_full.shape))
print("test set label shape: {}".format(y_test_full.shape))

#### Keep data from 5 to 9 included

In [ ]:
# Find indices of labels 5 to 9
train_index = np.squeeze(np.argwhere(y_train_full>=5))
test_index = np.squeeze(np.argwhere(y_test_full>=5))

In [ ]:
print(train_index.shape)
print(test_index.shape)

In [ ]:
(x_train, y_train) = (x_train_full[train_index], y_train_full[train_index])
(x_test, y_test) = (x_test_full[test_index], y_test_full[test_index])

#### Preprocessing

- normalise image values
- binarise categorical labels

In [ ]:
x_train_n = np.expand_dims(x_train/255, axis=-1).astype('float32')
x_test_n = np.expand_dims(x_test/255, axis=-1).astype('float32')

num_classes = 5

# convert class vectors to binary class matrices
y_train_b = to_categorical(y_train-5, num_classes)
y_test_b = to_categorical(y_test-5, num_classes)

In [ ]:
print("training set shape [labels 5 to 9]: {}".format(x_train_n.shape))
print("training set label shape [labels 5 to 9]: {}".format(y_train_b.shape))
print("test set shape [labels 5 to 9]: {}".format(x_test_n.shape))
print("test set label shape [labels 5 to 9]: {}".format(y_test_b.shape))

In [ ]:
pick_and_show_image(x_train, y_train);
pick_and_show_image(x_train_n, y_train_b);
pick_and_show_image(x_test, y_test);
pick_and_show_image(x_test_n, y_test_b);

## Train classification model

In [ ]:
model = tf.keras.Sequential(name="mnist529")
model.add(InputLayer((28, 28, 1), name="input"))
model.add(Conv2D(32, (3, 3), strides=(1, 1), padding="same", activation="relu", name="conv_1"))
model.add(MaxPool2D((2, 2), name="max_pool_1"))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding="same", activation="relu", name="conv_2"))
model.add(MaxPool2D((2, 2), name="max_pool_2"))
model.add(Dropout(.25, name="droput_025"))
model.add(Flatten(name="flatten"))
model.add(Dense(128, name="dense_1", activation="softmax"))
model.add(Dropout(.5, name="droput_05"))
model.add(Dense(5, name="dense_out", activation="softmax"))

In [ ]:
model.summary()

In [ ]:
optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
print('# Fit model on training data')
history = model.fit(x_train_n, y_train_b,
                    batch_size=32,
                    epochs=10,
                    validation_data=(x_test_n, y_test_b))

score = model.evaluate(x_test_n, y_test_b, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Save model


In [ ]:
if not "gan_evaluation" in os.listdir("./"):
    os.mkdir("gan_evaluation")
model.save("gan_evaluation/gan_evaluation_model.h5")

## Make prediction

In [ ]:
saved_model = tf.keras.models.load_model("gan_evaluation/gan_evaluation_model.h5")

In [ ]:
y_pred = saved_model.predict_classes(x_test_n)+5

In [ ]:
y_pred[0]

In [ ]:
y_prob = saved_model.predict(x_test_n)

In [ ]:
print(y_prob[0,:])
print(np.max(y_prob[0,:]))
print(np.argmax(y_prob[0,:])+5)

In [ ]:
np.argwhere((y_pred - y_test)==0).shape[0]/len(y_test)

In [ ]:
pick_and_show_image(x_test_n, y_pred);
pick_and_show_image(x_test_n, y_pred);
pick_and_show_image(x_test_n, y_pred);
pick_and_show_image(x_test_n, y_pred);

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
batch_size = 32
generated_images = x_test_n[:batch_size, :, :, :]
# predictions
y_prob = saved_model.predict(generated_images)
for i in range(batch_size):
    label = np.argmax(y_prob[i,:])+5
    prob = np.max(y_prob[i,:])
    # PLOT
    plt.subplot(4, 8, 1 + i)
    plt.axis('off')
    plt.imshow(generated_images[i, :, :, 0], cmap='gray_r')
    plt.title("L={}\nP={:.2f}".format(label, prob), size=10)
plt.savefig(os.path.join("gan_evaluation", "test"), dpi=200)
plt.close()
    
        